In [1]:
import json
from authorization import SpotifyAuth

In [2]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [3]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

In [4]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [5]:
import spotify_functions as spotify

In [6]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [7]:
len(history)

3

### Fetch details of a song

#### See available data for a song

In [8]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [9]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [10]:
history[0]['played_at']

'2024-08-17T15:11:38.535Z'

In [11]:
print(history[0]['context'])

None


# Extract, Transform, Load (ETL)

## Dimensions

### DIM_USER

In [12]:
from dw_etl_scripts.dim import *

#### Load (Type 0 SCD)

In [13]:
# dim_user_load(spotify, spotify_authorisation.get_access_token())

### DIM_ARTIST

#### Load (Type 1 SCD)

In [14]:
dim_artist_load(spotify, spotify_authorisation.get_access_token(), cdc_time)

### DIM_ALBUM

In [15]:
dim_album_load(spotify, spotify_authorisation.get_access_token(), cdc_time)

In [120]:
album_list = [play['track']['album'] for play in history]

In [122]:
df_album = pd.DataFrame(album_list)
df_album['cover_image_url'] = df_album['images'].apply(get_url)
df_album = df_album.drop(columns=['href','uri','external_urls', 'images','artists','type'])
df_album = df_album.drop_duplicates()
df_album = df_album.rename(columns={'id':'album_id','name':'album_name'})
# df_album

#### Load (Type 1 SCD)

In [123]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Text, Date

metadata = MetaData()

dim_album = Table(
    'DIM_ALBUM', metadata,
    Column('album_wid', Integer, primary_key=True, autoincrement=True),
    Column('album_id', String(62), nullable=False, unique=True),
    Column('album_type', String(20), nullable=False),
    Column('album_name', Text, nullable=False),
    Column('release_date', Date),
    Column('release_date_precision', String(20)),
    Column('total_tracks', Integer),
    Column('cover_image_url', Text)
)

In [124]:
sql_functions.save_to_sql(df_album, dim_album, key_cols = ['album_id'], wid = ['album_wid'], mode = 'upsert')

### BRIDGE_ALBUM_ARTIST

In [125]:
dim_artist = sql_functions.read_from_sql('DIM_ARTIST')
dim_album = sql_functions.read_from_sql('DIM_ALBUM')
df_album = pd.DataFrame(album_list)
df_album_map = df_album[['id','artists']]
df_album_map = df_album_map.explode('artists')
df_album_map['artist_id'] = df_album_map['artists'].apply(lambda x:x['id'])
df_album_map = df_album_map.drop(columns=['artists'])
# df_album_map

In [126]:
# Join with DIM_ARTIST, DIM_ALBUM to get ARTIST_WID, ALBUM_WID
df_bridge_album_artist = df_album_map.merge(dim_artist, left_on='artist_id', right_on='spotify_id')
df_bridge_album_artist = df_bridge_album_artist.merge(dim_album, left_on='id', right_on='album_id')
df_bridge_album_artist = df_bridge_album_artist[['album_wid','artist_wid']]
df_bridge_album_artist = df_bridge_album_artist.drop_duplicates()
# df_bridge_album_artist

#### Load (Type 1 SCD)

In [127]:
from sqlalchemy import MetaData, Table, Column, Integer

metadata = MetaData()

bridge_album_artist = Table(
    'BRIDGE_ALBUM_ARTIST', metadata,
    Column('album_artist_map_wid', Integer, primary_key=True, autoincrement=True),
    Column('album_wid', Integer, nullable=False),
    Column('artist_wid', Integer, nullable=False)
)

In [128]:
sql_functions.save_to_sql(
    df_bridge_album_artist
    , bridge_album_artist
    , key_cols = ['album_wid', 'artist_wid']
    , wid = ['album_artist_map_wid']
    , mode = 'ignore'
)

### DIM_SONG

In [129]:
dim_album = sql_functions.read_from_sql('DIM_ALBUM')

In [130]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [131]:
track_list = [play['track'] for play in history]

In [132]:
df_songs = pd.DataFrame(track_list)
df_songs = df_songs.explode('artists')
df_songs['artist_id'] = df_songs['artists'].apply(lambda x: x['id'])
df_songs['isrc_id'] = df_songs['external_ids'].apply(lambda x: x['isrc'])
df_songs = df_songs.drop(columns=['href','uri','external_urls','artists','external_ids', 'artist_id', 'popularity'])
df_songs['album_id'] = df_songs['album'].apply(lambda x:x['id'])
df_songs = df_songs.merge(dim_album[['album_wid','album_id']], left_on='album_id', right_on='album_id')
df_songs = df_songs.drop(columns=['album','album_id'])
df_songs = df_songs.rename(columns={'id':'song_id'})
df_songs = df_songs.drop_duplicates()
df_songs

,disc_number,duration_ms,explicit,song_id,is_local,name,preview_url,track_number,type,isrc_id,album_wid
0,1,149706,False,6XQHlsNu6so4PdglFkJQRJ,False,"City Of Stars - From ""La La Land"" Soundtrack",https://p.scdn.co/mp3-preview/fde76ee19ffd4123...,9,track,USUG11600656,71
2,1,235813,False,72R0X0h8YaxYNpegeoOl0M,False,Kiss Me Slowly,https://p.scdn.co/mp3-preview/6de18c73982456bc...,7,track,USUM71100714,72
4,1,206218,False,4ENjopIjOD5PKPb52NKNKo,False,Hero,https://p.scdn.co/mp3-preview/fea431c768d0d137...,1,track,QZXDB2300087,60
5,1,87000,False,6me7F0aaZjwDo6RJ5MrfBD,False,Evergreen,https://p.scdn.co/mp3-preview/fca7555fba8261b7...,1,track,SEYOK1732035,47
6,1,252818,False,456WNXWhDwYOSf5SpTuqxd,False,Dog Days Are Over,https://p.scdn.co/mp3-preview/f2e9f824fbfb697b...,1,track,GBUM70900209,66


#### Load (Type 1 SCD)

In [133]:
from sqlalchemy import BigInteger, Boolean, Text


metadata = MetaData()

dim_song = Table(
    'DIM_SONG', metadata,
    Column('song_wid', Integer, primary_key=True, autoincrement=True),
    Column('song_id', String(62), nullable=False, unique=True),
    Column('album_wid', Integer),
    Column('disc_number', Integer),
    Column('duration_ms', BigInteger, nullable=False),
    Column('explicit', Boolean),
    Column('is_local', Boolean),
    Column('name', Text, nullable=False),
    Column('preview_url', Text),
    Column('track_number', Integer),
    Column('type', String(50)),
    Column('isrc_id', String(100))
)

In [134]:
sql_functions.save_to_sql(df_songs, dim_song, key_cols = ['song_id'], wid = ['song_wid'], mode = 'upsert')

### FACT_PLAY

In [165]:
dim_song = sql_functions.read_from_sql('DIM_SONG')
df_play = pd.DataFrame(history)
df_play['song_id'] = df_play['track'].apply(lambda x:x['id'])
df_play = df_play.merge(dim_song, 'left', on = 'song_id')
df_play['context_playlist'] = df_play['context'].apply(lambda x:x['href'] if x and 'href' in x.keys() else None)
df_play = df_play[['song_wid','played_at','context_playlist']]
df_play['played_at'] = pd.to_datetime(df_play['played_at'])
df_play['played_at'] = df_play['played_at'] + pd.Timedelta(hours=5, minutes=30)
df_play['played_at'] = df_play['played_at'].apply(lambda x:x.strftime('%Y-%m-%d %H:%M:%S.%f'))
df_play

,song_wid,played_at,context_playlist
0,89,2024-08-11 16:33:43.891000,https://api.spotify.com/v1/playlists/1TkXrIRsO...
1,90,2024-08-11 16:31:13.712000,https://api.spotify.com/v1/playlists/1TkXrIRsO...
2,90,2024-08-11 16:30:54.486000,https://api.spotify.com/v1/playlists/1TkXrIRsO...
3,50,2024-08-11 16:13:44.672000,None
4,36,2024-08-11 12:12:53.419000,None
5,87,2024-08-11 12:03:50.717000,None


#### Load

In [166]:
from sqlalchemy import Table, Column, Integer, String, Text, DateTime

# Define the metadata
metadata = MetaData()

# Define the FACT_PLAY table
fact_play = Table(
    'FACT_PLAY', metadata,
    Column('play_wid', Integer, primary_key=True, autoincrement=True),
    Column('song_wid', Integer, nullable=False),
    Column('played_at', DateTime, nullable=False),
    Column('context_playlist', Text, nullable=False)
)

In [169]:
sql_functions.save_to_sql(df_play, fact_play, key_cols = ['song_wid','played_at'], wid = ['play_wid'], mode = 'append')